# Laboratorio: Analisi Flussi di Popolazione con pandas

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
`nome.cognome@unibo.it`

In [ ]:
# test librerie
import numpy
import pandas
print(numpy.__version__, pandas.__version__)

## Caso di studio: Analisi flussi tra stati USA

- L'ufficio del censimento degli Stati Uniti d'America pubblica annualmente un riepilogo dei flussi della popolazione tra stati
- I dataset per ogni anno in formato XLS (Excel) possono essere reperiti alla pagina https://www.census.gov/data/tables/time-series/demo/geographic-mobility/state-to-state-migration.html
- In questa esercitazione, lavoriamo sui dati dei flussi tra gli stati relativi al 2016
- Eseguire la seguente cella per scaricare il file con i dati dell'esercitazione, se non presente

In [ ]:
import os.path
if not os.path.exists("usa_census.npz"):
    from urllib.request import urlretrieve
    urlretrieve("https://git.io/vxh8Y", "usa_census.npz")

## Caricamento dei dati

- Abbiamo raccolto i dati da analizzare in un _archivio NumPy_, un file contenente un insieme di array con nomi associati
- Usiamo la funzione `load` di NumPy per caricarlo
  - `allow_pickle=True` abilita il caricamento di oggetti Python arbitrari, in questo caso serve per le stringhe con i nomi degli stati, **usarlo solo su file da fonti fidate!**

In [ ]:
import numpy as np
data = np.load("usa_census.npz", allow_pickle=True)

- L'oggetto `data` ottenuto è simile ad un dizionario, i cui valori sono le matrici e le cui chiavi sono i loro nomi
- Otteniamo una lista dei nomi delle matrici caricate

In [ ]:
print(", ".join(data.keys()))

## Dati disponibili

I dati contenuti nel file sono i seguenti:
- `states`: vettore con i nomi degli stati
- `population`: vettore con la popolazione totale attuale di ogni stato
- `area`: vettore con la superficie di terra di ciascuno stato in miglia quadrate
- `same_house`: vettore con numero di abitanti per stato che non hanno cambiato residenza nell'ultimo anno
- `same_state`: vettore con numero di abitanti per stato che hanno cambiato residenza nell'ultimo anno, ma non lo stato
- `other_state`: vettore con numero di abitanti per stato che si sono trasferiti da un altro stato nell'ultimo anno
- `state_to_state`: matrice col numero di abitanti trasferitisi nell'ultimo anno da ciascuno stato ad un altro
- `from_abroad`: vettore con numero di abitanti per stato trasferitisi negli USA dall'estero nell'ultimo anno

- Possiamo stampare nome, tipo e forma di ciascun array

In [ ]:
# modo semplice: passo i valori a print che li separa con uno spazio
for name, array in data.items():
    print(name, array.dtype, array.shape)

In [ ]:
# per maggiore leggibilità posso usare una f-string
for name, array in data.items():
    print(f"{name:>15}: {array.dtype!s:>8} {array.shape}")
    # "!s" = converti in stringa con str(x) in modo da poter applicare formato
    # ">N" = riserva N caratteri e allinea a destra

- Vediamo ad esempio l'array con i nomi degli stati, che sono in ordine alfabetico

In [ ]:
data["states"]

- In tutti i vettori, i dati relativi ad uno **stesso stato** si trovano nella **stessa posizione**
  - ad es. i dati relativi all'Alabama (`states[0]`) si trovano nella posizione 0 di ciascun array (`population[0]`, `area[0]`, ...)

- Per comodità, importiamo gli array su cui lavoreremo come variabili locali

In [ ]:
states = data["states"]
population = data["population"]
area = data["area"]

- In questo modo potremo usare ad es. `states` al posto di `data["states"]`

In [ ]:
states   [:5]

In [ ]:
population   [:5]

## Esercizio 1: Ripasso NumPy

Usando sugli array `states` e `population` le operazioni viste nello scorso laboratorio, estrarre:
- **(1a)** il numero di abitanti del 5° stato in ordine alfabetico (California)
- **(1b)** i nomi degli ultimi tre stati in ordine alfabetico
- **(1c)** il numero di abitanti in Florida (senza sapere a priori la sua posizione)
- **(1d)** i nomi degli stati con almeno 20 milioni di abitanti
- **(1e)** il numero totale di abitanti in tutti gli stati
- **(1f)** il nome dello stato con meno abitanti

In [ ]:
population[4]

In [ ]:
states[-3:]

In [ ]:
population[states == "Florida"][0]

In [ ]:
states[population >= 2e7]

In [ ]:
population.sum()

In [ ]:
print(states[population == population.min()])
print(states[population.argmin()])
print(states[population == population.max()])
print(states[population.argmax()])

## pandas

- **pandas** è una libreria Python di uso comune per lavorare con dati in forma tabulare
- Definisce _serie_ e _data frame_, strutture dati simili agli array di NumPy ma dotate di _indici_ che etichettano i dati
- Questo rende semplice reperire dati specifici, ad esempio la popolazione di uno stato data l'etichetta (il nome)
- Iniziamo importando il package `pandas` con l'alias convenzionale `pd`

In [ ]:
import pandas as pd

## Serie

- Una **_serie_** pandas è un vettore di valori simile ad un array 1D NumPy con un'etichetta associata a ciascuno
- La sequenza di etichette costituisce l'_indice_ della serie, anch'esso un array 1D
- Il costruttore `Series` permette di creare una serie di cui sono forniti i dati e l'indice (`index`) in due vettori di uguale lunghezza
- Creiamo la serie `population` che sostituisca l'omonimo array NumPy
  - i dati della serie sono contenuti nell'array stesso
  - le etichette sono i nomi degli stati contenuti nell'array `states`

In [ ]:
population = pd.Series(data["population"], index=data["states"])

- Visualizziamo la serie, utilizzando il metodo `head` per estrarne solo i primi N elementi

In [ ]:
population.head(7)

- Sulla sinistra vediamo le **etichette** che formano l'indice: i nomi degli stati
- Sulla destra vediamo i **valori**: la popolazione di ciascuno stato
- In fondo vediamo il datatype dei valori (numeri interi)

## Attributi di base di una serie

- Una serie è composta dai dati e dalle etichette: gli attributi `values` e `index` restituiscono questi elementi in forma di vettori

In [ ]:
population.values   [:5]

In [ ]:
population.index    [:5]

- Il numero di valori si può ottenere con la funzione `len` di Python o (per analogia con gli array) con l'attributo `size`

In [ ]:
print(len(population))
# oppure
print(population.size)

## Selezionare dati da una serie

- Le serie supportano in pratica le stesse tecniche di selezione degli array, ma usando le **etichette al posto degli indici numerici**
- Per cui ad es. per ottenere la popolazione della California si scrive semplicemente:

In [ ]:
population["California"]

In [ ]:
# Conversione esplicita da serie a dizzionario python
dict(population)

- Se si seleziona un'etichetta inesistente, si ha un `KeyError` come per i dizionari `{...}`

In [ ]:
population["California"]

In [ ]:
population["C" : "M"]

- Si può selezionare un intervallo tra due etichette
  - sono **inclusi entrambi gli estremi**, a differenza di altre strutture dati

In [ ]:
population["Arizona":"Colorado"]

- Gli estremi dell'intervallo _A:B_ possono non esistere, vengono comunque presi gli elementi inclusi con _A <= etichetta <= B_
  - funziona se le etichette sono disposte in ordine, altrimenti si ha errore!
- Ad es. per selezionare gli stati con iniziali da S a U (incluse) selezioniamo le etichette nell'intervallo `"S":"V"`
  - con `"S":"U"` verrebbe escluso "Utah", perché tale stringa è maggiore di "U"
  - Posso farlo perchè le stringhe sono combarabili (si da per buono l'ordine)

In [ ]:
population["S":"V"]

- Si può selezionare un sottoinsieme della serie indicando una lista di etichette

In [ ]:
# lista di stati della costa ovest
west_coast = ["Washington", "Oregon", "California"]

In [ ]:
# popolazione stati della costa ovest
population[west_coast]

In [ ]:
population[["Washington", "Oregon", "California"]] # l'ordine è quello della lista interna degli indici

## Creazione delle altre serie

- Creiamo una serie per altri vettori di dati, utilizzando sempre il vettore di nomi degli Stati `states` come indice
  - _(se servissero, gli array NumPy originali rimangono disponibili nell'oggetto `data`)_

In [ ]:
area        = pd.Series(data["area"],        index=data["states"])
other_state = pd.Series(data["other_state"], index=data["states"])
from_abroad = pd.Series(data["from_abroad"], index=data["states"])

## Operazioni tra serie

- Come per gli array, è possibile effettuare operazioni elemento per elemento tra due serie o tra una serie ed un valore singolo
- Si possono utilizzare operazioni binarie (`+`, `*`, ...) e funzioni universali di NumPy (`np.log`, ...)
- Nel caso di due serie, le operazioni sono applicate **tra elementi di uguale etichetta** (indipendentemente dalla posizione!)
  - la serie risultante avrà valori mancanti (NA) in corrispondenza di etichette presenti in un solo operando

- Ad es. per ottenere la popolazione in milioni di abitanti

In [ ]:
(population / 1_000_000)    .head(3)

- Per ottenere il logaritmo in base 10 della popolazione (ad es. per creare un grafico in tale scala):

In [ ]:
np.log10(population)   .head(3)

## Esercizio 2: Operazioni tra serie

- **(2a)** La serie `area` riporta la superficie degli stati in miglia quadrate: ricavare una serie `area_km2` con la superficie in chilometri quadrati (1 mi² = 2,59 km²)
- **(2b)** Creare una serie `density` con la densità di popolazione di ciascuno stato in abitanti per km²

In [ ]:
area_km2 = area * 2.59

In [ ]:
density = population / area_km2

## Serie booleane e selezione per condizioni

- Applicando una comparazione ad una serie, si ottiene una serie di valori booleani
- Ad esempio, per ottenere la serie che associa `True` agli stati non più grandi di 5.000 miglia quadrate:

In [ ]:
is_small = area <= 5000
is_small.head(10)

- Come per gli array, una serie booleana può essere usata per selezionare solamente alcuni elementi di una serie di dati
  - sono selezionati solamente gli elementi alla cui etichetta nella serie booleana è associato `True`
- Ad esempio, per mostrare la superficie dei soli stati piccoli individuati sopra:

In [ ]:
area[area <= 5000]
# oppure: area[is_small]

- La serie da cui sono selezionati i dati e quella usata come condizione possono anche differire...

In [ ]:
# popolazione degli stati piccoli
population[area <= 5000]

- Possiamo combinare serie booleane con gli operatori `&` (AND), `|` (OR), `^` (XOR), `~` (NOT)
  - tali operatori hanno normalmente priorità superiore alle comparazioni (`>`, `<=`, ...), **usare le parentesi** per evitare errori

In [ ]:
# stati piccoli con più di un milione di abitanti
population[(population >= 1_000_000) & (area <= 5000)]

## Operazioni di riduzione sulle serie

- Le serie offrono metodi simili a quelli degli array per aggregare i dati: `sum`, `mean`, `min`, `max`, ...
- Ad esempio, per ottenere la popolazione totale di tutti gli Stati:

In [ ]:
population.sum()

- Per ottenere la popolazione nello Stato dove è maggiore:

In [ ]:
population.max()

- Per conoscere a quale Stato corrisponde, uso il metodo `idxmax` che restituisce l'etichetta del valore maggiore

In [ ]:
population.idxmax()

- Sono definiti analogamente `min` e `idxmin` per il valore minimo e la sua etichetta

In [ ]:
population.min()

In [ ]:
population.idxmin()

## Esercizio 3: Operazioni di riduzione su serie

Ricavare:
- **(3a)** la densità di popolazione dello stato più piccolo
- **(3b)** il numero di stati la cui popolazione è superiore al milione di abitanti
- **(3c)** il totale della popolazione degli stati sulla costa ovest (usare lista `west_coast` definita sopra)
- **(3d)** la densità media degli stati con almeno 10 milioni di abitanti

In [ ]:
density[area.idxmin()]

In [ ]:
print(len(list(population[population > 1e6]))) # operazioni di selezione e cast a lista inutili
print(len(population[population > 1e6])) # operazione di selezione inuitle
print((population > 1e6).sum())
print(f"media : {(population > 1e6).mean()}")

In [ ]:
population[west_coast].sum()

In [ ]:
density[population > 1e7].mean()

## DataFrame

- Un `DataFrame` pandas contiene dati strutturati in **forma tabulare**, dove in genere
  - **ogni riga** della tabella rappresenta **un elemento** dell'insieme che si sta analizzando (una persona, un territorio, ...)
  - **ogni colonna** rappresenta **una caratteristica** di interesse (_feature_) degli elementi (età, popolazione, ...)
- Le colonne di un data frame costituiscono un insieme di serie tutte con la stessa sequenza di etichette
  - tale sequenza costituisce l'_indice delle righe_ del frame (o semplicemente _indice_)
  - ogni colonna ha un nome, i nomi delle colonne formano l'_indice delle colonne_ del frame
  - i datatype dei valori possono differire da una colonna all'altra

## Creare un DataFrame

- Per creare un DataFrame possiamo passare un dizionario con una serie di valori per ciascuna colonna, la cui chiave è il nome
  - l'indice delle serie, identico per tutte, è usato come indice delle righe del DataFrame
  - se alcune etichette fossero presenti solo in alcune serie, si otterrebbero valori mancanti nelle altre colonne

In [ ]:
census = pd.DataFrame({
    "population": population,
    "from_abroad": from_abroad,
    "area": area_km2
})

- Visualizziamo le prime righe del DataFrame come esempio...

In [ ]:
census.head(5)

- In alto sono scritti i nomi delle colonne, che costituiscono _l'indice delle colonne_
- A sinistra sono scritti i nomi degli Stati, che costituiscono _l'indice delle righe_
- Gli indici **non** contano come righe o colonne del frame

- Da un `DataFrame` possiamo estrarre i suoi componenti:
  - la matrice dei dati `values`
  - l'indice delle righe `index`
  - l'indice delle colonne `columns`

In [ ]:
census.values   [:4] # prime 4 righe

In [ ]:
census.index   [:4] # primi 4 stati

In [ ]:
census.columns

- Come per le matrici, possiamo ottenere il numero di righe e di colonne con `shape`

In [ ]:
census.shape

- Un modo alternativo per creare un DataFrame è passare una matrice (array 2D) con i dati e vettori con nomi di righe e colonne
- Ad esempio, possiamo convertire in DataFrame la matrice `state_to_state` che indica il numero di persone trasferitesi da uno stato all'altro nell'ultimo anno
- Creiamo il DataFrame passando i dati nella matrice e usando il vettore dei nomi `states` come indice sia delle righe (`index`) che delle colonne (`columns`)

In [ ]:
state_to_state = pd.DataFrame(data["state_to_state"], index=data["states"], columns=data["states"])

- Abbiamo così convertito la matrice in una tabella leggibile, di cui quì visualizziamo una parte _(vedremo a breve come funziona `iloc`)_

In [ ]:
state_to_state   .iloc[:5, :5]

- Ogni riga rappresenta lo stato di destinazione, ogni colonna lo stato d'origine
  - ad esempio, 423 persone si sono trasferite dall'Alabama all'Alaska

## Estrarre ed aggiungere colonne

- Il DataFrame è utilizzabile come un dizionario i cui valori sono le colonne e le chiavi i loro nomi
- Selezionando una chiave, viene restituita la colonna con quel nome in forma di serie
  - l'indice della serie restituita è l'indice del DataFrame

In [ ]:
census["population"]   .head(3)

- Seguendo la stessa logica, è possibile aggiungere colonne assegnando una serie di valori ad un nome
  - se una colonna col nome dato esiste già, viene sovrascritta
  - se viene passata una serie, i valori sono assegnati per corrispondenza tra etichette e indice delle righe
  - se viene passato un valore singolo (scalare), questo è replicato in tutte le righe
- Questo permette di creare facilmente colonne con valori derivati dalle altre
- Ad esempio, per aggiungere una colonna con la densità di popolazione _(senza usare la serie creata in precedenza)_:

In [ ]:
census["density"] = census["population"] / census["area"]

- La colonna viene così aggiunta a destra nel DataFrame:

In [ ]:
census.head(5)

## Statistiche sui dati

- Anche sui `DataFrame` sono disponibili i metodi `sum`, `mean`, `min`, ... per aggregare i dati
- Di default, restituiscono una serie con la statistica calcolata **colonna per colonna**

In [ ]:
# medie su tutti gli stati
census.mean()

In [ ]:
# somme su tutti gli stati
census.sum()

- Si noti che non sempre tutte le statistiche che si possono estrarre sono significative!
  - ad esempio la somma ha senso sulla popolazione e sull'area, ma non sulla densità

- Il metodo `describe` (applicabile anche alle serie) fornisce rapidamente un'insieme di statistiche sui valori di ciascuna colonna, utili ad analizzarne la distribuzione

In [ ]:
census.describe()

La tabella ottenuta mostra:
- `count` = valori non mancanti, ovvero diversi da NA
  - in questo caso non ci sono valori mancanti, quindi tutti i count sono pari al numero di righe (51)
- `mean` = media
  - ad es. la popolazione media per stato è di 6,26 milioni di abitanti
- `std` = deviazione standard
- `min`/`max` = valori minimo/massimo
- `25%`/`50%`/`75%` = percentili
  - ad es. il 25% degli stati ha densità di popolazione **inferiore a** 18,46 abitanti per km²

- Di default le statistiche sono calcolate per colonne (riducendo le righe), perché è l'esigenza più comune
  - nella tabella `census`, così come in altri casi pratici, ogni colonna ha valori in scale diverse per cui non ha senso calcolare somma, media, ecc. per riga
  - `sum()` in un DataFrame corrisponde in pratica a `sum(0)` in una matrice NumPy
- Nel caso si vogliano calcolare le statistiche per righe (riducendo le colonne), si può specificare `axis=1`
  - questo corrisponde in pratica a `sum(1)` su una matrice NumPy

- Ad esempio, applicando `sum()` in modo standard al DataFrame `state_to_state`, otteniamo il totale di abitanti trasferitisi **da** ogni stato (colonne)...

In [ ]:
state_to_state.sum()   .head(3)

- ...mentre applicando `sum(axis=1)` otteniamo il totale di abitanti trasferitisi **verso** ogni stato (righe)...

In [ ]:
state_to_state.sum(axis=1)   .head(3)

- ...che è esattamente quanto riportato nella serie `other_state`: possiamo verificarlo col metodo `equals` che indica se due serie sono identiche

In [ ]:
state_to_state.sum(axis=1).equals(other_state)

## Esercizio 4: Operazioni sui DataFrame

Utilizzando i frame `census` e `state_to_state` (senza utilizzare le serie usate in precedenza), ricavare

- **(4a)** la superficie dello stato più grande
- **(4b)** il numero totale di persone emigrate dall'Arizona ad un altro stato
- **(4c)** il nome dello stato verso cui sono immigrate meno persone dagli altri stati
  - promemoria per `state_to_state`: colonna = stato di origine, riga = stato di destinazione

In [ ]:
census["area"].idxmax()

In [ ]:
census["area"].max()

In [ ]:
state_to_state["Arizona"].sum()

In [ ]:
state_to_state.sum(axis=1).idxmin()

## Selezione

- Per selezionare una porzione di DataFrame, vanno indicate righe e colonne da includere
- Esistono diversi _selettori_, che consentono di selezionare parti di un DataFrame `X` in modo diverso:
  - `X.loc[...]` seleziona righe e colonne specificate **per etichetta** (es.: colonna "population")
  - `X.iloc[...]` seleziona righe e colonne specificate **per posizione** (es.: colonna 0)
- In entrambi i casi vanno specificate righe e colonne da selezionare come avviene nelle matrici
  - si può selezionare per valore singolo, intervallo, lista di valori o serie booleana con stesso indice
  - `:` indica di selezionare tutte le righe o le colonne, se usato per le colonne può essere omesso

- Ad esempio, abbiamo già visto sopra come selezionare solamente le prime righe e colonne di un DataFrame:

In [ ]:
state_to_state.iloc[:3, :5]  # forma breve per [0:3, 0:5]

- Selezionando una singola riga di un DataFrame, otteniamo una serie con i valori di tutte le colonne

In [ ]:
census.loc["California", :]   # ", :" può essere omesso

- Usando serie booleane, possiamo selezionare righe (o colonne) per condizioni

In [ ]:
# righe relative agli Stati con meno di 700.000 abitanti
census.loc[census["population"] < 700000]

In [ ]:
# sole densità di popolazione degli Stati con superficie minore di 10.000 km²
census.loc[census["area"] < 10000, "density"]

- Per selezionare righe per posizione e colonne per etichetta (o viceversa), si possono concatenare le applicazioni di `loc` e `iloc`

In [ ]:
# prime tre righe, colonna "population"
census.iloc[:3].loc[:, "population"]

- `loc` e `iloc` si possono usare con la stessa logica anche sulle serie (`loc` equivale alla selezione normale per etichette)

In [ ]:
# valori di population dal 6° al 10°
population.iloc[5:10]

## Ordinamento

- La funzione `sort_index` ordina le righe di un frame secondo le etichette
  - nel nostro caso le righe sono già ordinate alfabeticamente per etichetta
- `sort_values` invece ordina le righe secondo i valori di una o più colonne specificate
  - le colonne oltre alla prima si usano per risolvere i pareggi, come nella clausola `ORDER BY` di SQL
- Entrambi i metodi restituiscono una copia ordinata del frame senza modificarlo
  - specificando `inplace=True` invece si modifica il DataFrame originale e viene restituito `None`
- Possiamo ad esempio per visualizzare i 5 Stati più popolati
  - specifichiamo `ascending=False` per ottenere un ordinamento decrescente

In [ ]:
census.sort_values("population", ascending=False).head(5)

In [ ]:
# indicare più di una colonna di ordinamento serve per avere un metodo di ordinamento in caso di parità sulla prima 
census.sort_values(["population", "area"], ascending=False).head(5)

In [ ]:
census.sort_values(["population", "area"], ascending=False, inplace=True)
print(census)

- sort_values ritorna un nuova dataframes
- inplace lavora sulla stessa e restituisce None

## Esercizio 5: Selezione da DataFrame

Utilizzando il frame `census` (senza utilizzare le serie usate in precedenza), ricavare

- **(5a)** la superficie della California
- **(5b)** la popolazione (colonna 0) del 13° stato nella tabella
- **(5c)** la densità di popolazione dello stato con superficie maggiore
- **(5d)** la popolazione totale degli stati con nome che inizia per M
- **(5e)** la superficie complessiva degli stati con almeno 20 milioni di abitanti
- **(5f)** la popolazione media degli stati con almeno l'1% di popolazione immigrato dall'estero (`from_abroad`) nell'ultimo anno
- **(5g)** la superficie totale dei 5 stati con densità di popolazione minore
- **(5h)** la popolazione (colonna 0) del 3° stato con superficie maggiore

In [ ]:
census["area"]["California"]

In [ ]:
census.loc["C" : "Z", "area"]

In [ ]:
census.iloc[12, 0]

In [ ]:
census.index[12]

In [ ]:
census.loc[census["area"].idxmax(), "density"]

In [ ]:
census.loc["C": "M", "population"]

In [ ]:
census.loc[census["population"] > 2e7, "area"].sum()

In [ ]:
census.loc[census["from_abroad"] / census["population"] >= 0.01, "population"].mean()

In [ ]:
census.sort_values("density").head(5)["area"].mean()

In [ ]:
census.sort_values("area", ascending=False).iloc[2, 0]

## Creazione di grafici con matplotlib 

- I grafici sono in generale utili nell'analisi preliminare di dati per visualizzare come sono distribuiti i valori in una serie di dati
- **matplotlib** è tra le librerie Python più diffuse per la creazione di svariati tipi di grafici (a barre, a torta, a dispersione, ...)
- Iniziamo in questo laboratorio a vedere come creare alcune tipologie di grafici, vedremo nel prossimo altri tipi di grafici e opzioni più avanzate
- Vediamo due approcci alternativi per creare dei grafici con matplotlib
  - invocando direttamente le funzioni dell'interfaccia semplificata `pyplot` (che riprende quella di Matlab)
  - utilizzando i metodi `plot.*` forniti da pandas per creare rapidamente grafici da serie e DataFrame

- Per iniziare, importiamo l'interfaccia `pyplot` con l'alias convenzionale `plt`

In [ ]:
import matplotlib.pyplot as plt

- Invochiamo inoltre la seguente cella per assicurarci che i grafici siano disegnati all'interno dei file Jupyter (necessario in versioni più vecchie di Jupyter)

In [ ]:
%matplotlib inline

## Grafici a barre

- Un grafico a barre raffigura una serie di valori come barre di diversa altezza, consentendone un confronto rapido
- Si crea con la funzione `bar`, indicando in ordine le etichette da assegnare alle barre e i rispettivi valori da usare come altezze

In [ ]:
plt.bar(
    [ "two", "four",   "pi",    "e"],  # etichette
    [     2,      4,   3.14,   2.71]   # valori
)

- Possiamo utilizzarli ad esempio per visualizzare i valori di una serie
- Prendiamo come esempio la popolazione dei paesi della costa ovest
- Essendo numeri molto grandi, matplotlib li visualizza in notazione scientifica ("1e7" = decine di milioni)

In [ ]:
west_coast_population = population[west_coast]
west_coast_population

In [ ]:
plt.bar(
    west_coast_population.index,  # etichette
    west_coast_population.values  # valori
);  # <-- aggiungere ";" alla fine per sopprimere l'output testuale

- Come alternativa più semplice, si può usare il metodo `plot.bar` disponibile sulle strutture dati pandas

In [ ]:
west_coast_population.plot.bar();

- Nei grafici successivi visualizziamo la popolazione di tutti i 51 stati in milioni di abitanti (dividiamo tutti i valori per 1.000.000), in modo da leggere più facilmente l'asse y

In [ ]:
population_mln = population / 1_000_000

- Tramite la funzione `figure` è possibile configurare alcune opzioni del grafico costruito con le istruzioni successive
  - in particolare possiamo regolare le dimensioni del grafico `figsize` in forma di tupla `(larghezza, altezza)`
- Utilizziamola creare un grafico più largo con la popolazione di tutti gli stati

In [ ]:
plt.figure(figsize=(20, 4))
population_mln.plot.bar();

- L'opzione `figsize` può anche essere passata direttamente a `plot.bar`

In [ ]:
population_mln.plot.bar(figsize=(20, 4));

- Con la funzione `grid` è possibile aggiungere una griglia al grafico su entrambi gli assi o su un solo asse ("x" o "y") indicato
  - per aggiungerla ad entrambi gli assi usare `plt.grid()` oppure specificare `grid=True` in `plot.bar`

In [ ]:
population_mln.plot.bar(figsize=(20, 4))
plt.grid(axis="y");

- Con le funzioni `title`, `xlabel` e `ylabel` è possibile specificare un titolo del grafico, un'etichetta per l'asse X e una per l'asse Y

In [ ]:
population_mln.plot.bar(figsize=(20, 4))
plt.grid(axis="y")
plt.title("Popolazione stati USA nel 2016")
plt.xlabel("Stato")
plt.ylabel("Popolazione (milioni di abitanti)");

## Box plot

- Un _box plot_ visualizza in modo compatto le statistiche di base di una o più serie di dati
- In pratica fornisce visivamente molte delle informazioni fornite dal metodo `describe` visto sopra
- Vediamolo ad esempio sulla popolazione degli stati usando la funzione `boxplot`...

In [ ]:
plt.figure(figsize=(4, 6))
plt.boxplot(population_mln, showmeans=True)
plt.grid(axis="y");

- ...o alternativamente il metodo `plot.box`

In [ ]:
population_mln.plot.box(showmeans=True, figsize=(4, 6))
plt.grid(axis="y");

- Gli estremi inferiore e superiore del rettangolo (circa 2 e 7 milioni di abitanti) sono il **primo e terzo quartile** ($Q_1$ e $Q_3$) dei dati, visualizzati come "25%" e "75%" nell'output di `describe`
  - il rettangolo rappresenta in pratica il 50% "centrale" dei valori
  - la distanza dal primo al terzo quartile ($Q_3-Q_1$) è detta _interquartile range_ (IQR)
- La linea centrale (circa 4 milioni) indica la **mediana** ("50%" in `describe`)
- I cerchi rappresentano i valori molto distanti dalla mediana, detti **outlier**
  - nello specifico, sono outlier tutti i valori che sono distanti dal quartile più vicino più di 1,5 volte l'IQR
  - formalmente, un valore $x$ è outlier se $x < Q_1 - 1.5\cdot{IQR}$ oppure $x > Q_3 + 1.5\cdot{IQR}$
  - 1,5 è un valore usato convenzionalmente, che può essere cambiato col parametro `whis` di `boxplot`/`box`
- I "baffi" (_whiskers_) indicano il **minimo** e il **massimo** dei soli dati "ordinari", ovvero escludendo gli outlier
- Il triangolo (circa 6 milioni) indica la **media**, che viene omessa se non si specifica `showmeans=True`
- Si riporta sotto l'output di `describe` sulla medesima serie, da cui ritornano i valori evidenziati nel box plot

In [ ]:
population_mln.describe()

- Dagli outlier nella parte superiore e dalla media superiore alla mediana, si intuisce ad occhio che la serie `population` ha pochi valori molto superiori alla mediana
- Per confronto, si osservi sotto il box plot di un array di 1.000 valori casuali con distribuzione normale: il grafico tende ad essere simmetrico (media = mediana, outlier distribuiti equamente)

In [ ]:
np.random.seed(123)
random_values = np.random.normal(size=1000)
plt.figure(figsize=(4, 6))
plt.boxplot(random_values, showmeans=True)
plt.grid(axis="y");

- Invocando `plot.box` su un DataFrame viene generato un grafico unico con i box plot di tutte le colonne numeriche
  - può essere difficilmente leggibile se le variabili hanno scale molto diverse

In [ ]:
census.plot.box(showmeans=True)
plt.grid(axis="y");

In [ ]:
# grafico solo su due colonne, parzialmente più leggibile
census[["from_abroad", "area"]].plot.box(showmeans=True)
plt.grid(axis="y");